# TRANSFORMERS 2

<div class="ab ca"><div class="ch bg dx dy dz ea"><p id="1daa" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">This is the second article in my series on Transformers. In the first article, we learned about the functionality of Transformers, how they are used, their high-level architecture, and their advantages.</p><p id="7505" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">In this article, we can now look under the hood and study exactly how they work in detail. We’ll see how data flows through the system with their actual matrix representations and shapes and understand the computations performed at each stage.</p><p id="6d02" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Here’s a quick summary of the previous and following articles in the series. My goal throughout will be to understand not just how something works but why it works that way.</p><ol class=""><li id="0d3b" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt nx ny nz bj" data-selectable-paragraph=""><a class="af mx" rel="noopener" target="_blank" href="/transformers-explained-visually-part-1-overview-of-functionality-95a6dd460452">Overview of functionality</a><strong class="na fh"> </strong><em class="oa">(How Transformers are used, and why they are better than RNNs. Components of the architecture, and behavior during Training and Inference)</em></li><li id="0b6f" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt nx ny nz bj" data-selectable-paragraph=""><strong class="na fh">How it works — this article</strong> <em class="oa">(Internal operation end-to-end. How data flows and what computations are performed, including matrix representations)</em></li><li id="8e85" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt nx ny nz bj" data-selectable-paragraph=""><a class="af mx" rel="noopener" target="_blank" href="/transformers-explained-visually-part-3-multi-head-attention-deep-dive-1c1ff1024853">Multi-head Attention</a> <em class="oa">(Inner workings of the Attention module throughout the Transformer)</em></li><li id="f77e" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt nx ny nz bj" data-selectable-paragraph=""><a class="af mx" rel="noopener" target="_blank" href="/transformers-explained-visually-not-just-how-but-why-they-work-so-well-d840bd61a9d3">Why Attention Boosts Performance</a> <em class="oa">(Not just what Attention does but why it works so well. How does Attention capture the relationships between words in a sentence)</em></li></ol><p id="f8c4" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">And if you’re interested in NLP applications in general, I have some other articles you might like.</p><ol class=""><li id="67b6" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt nx ny nz bj" data-selectable-paragraph=""><a class="af mx" rel="noopener" target="_blank" href="/foundations-of-nlp-explained-visually-beam-search-how-it-works-1586b9849a24">Beam Search</a> <em class="oa">(Algorithm commonly used by Speech-to-Text and NLP applications to enhance predictions)</em></li><li id="50fa" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt nx ny nz bj" data-selectable-paragraph=""><a class="af mx" rel="noopener" target="_blank" href="/foundations-of-nlp-explained-bleu-score-and-wer-metrics-1a5ba06d812b">Bleu Score</a> (<em class="oa">Bleu Score and Word Error Rate are two essential metrics for NLP models</em>)</li></ol><h1 id="1fdf" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Architecture Overview</h1><p id="0811" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">As we saw in <a class="af mx" rel="noopener" target="_blank" href="/transformers-explained-visually-part-1-overview-of-functionality-95a6dd460452">Part 1</a>, the main components of the architecture are:</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*cfNpm7aDO4lD3e-Wkwgc1g.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*cfNpm7aDO4lD3e-Wkwgc1g.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*cfNpm7aDO4lD3e-Wkwgc1g.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*cfNpm7aDO4lD3e-Wkwgc1g.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*cfNpm7aDO4lD3e-Wkwgc1g.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*cfNpm7aDO4lD3e-Wkwgc1g.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*cfNpm7aDO4lD3e-Wkwgc1g.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*cfNpm7aDO4lD3e-Wkwgc1g.png 640w, https://miro.medium.com/v2/resize:fit:720/1*cfNpm7aDO4lD3e-Wkwgc1g.png 720w, https://miro.medium.com/v2/resize:fit:750/1*cfNpm7aDO4lD3e-Wkwgc1g.png 750w, https://miro.medium.com/v2/resize:fit:786/1*cfNpm7aDO4lD3e-Wkwgc1g.png 786w, https://miro.medium.com/v2/resize:fit:828/1*cfNpm7aDO4lD3e-Wkwgc1g.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*cfNpm7aDO4lD3e-Wkwgc1g.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*cfNpm7aDO4lD3e-Wkwgc1g.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="521" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*cfNpm7aDO4lD3e-Wkwgc1g.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="8e44" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Data inputs for both the Encoder and Decoder, which contains:</p><ul class=""><li id="7a74" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph="">Embedding layer</li><li id="81b9" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">Position Encoding layer</li></ul><p id="f467" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Encoder stack contains a number of Encoders. Each Encoder contains:</p><ul class=""><li id="42f2" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph="">Multi-Head Attention layer</li><li id="1630" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">Feed-forward layer</li></ul><p id="9547" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Decoder stack contains a number of Decoders. Each Decoder contains:</p><ul class=""><li id="22cc" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph="">Two Multi-Head Attention layers</li><li id="e3b9" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">Feed-forward layer</li></ul><p id="2c2a" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Output (top right) — generates the final output, and contains:</p><ul class=""><li id="932b" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph="">Linear layer</li><li id="b599" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">Softmax layer.</li></ul><p id="454c" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">To understand what each component does, let’s walk through the working of the Transformer while we are training it to solve a translation problem. We’ll use one sample of our training data which consists of an input sequence (‘You are welcome’ in English) and a target sequence (‘De nada’ in Spanish).</p><h1 id="3594" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Embedding and Position Encoding</h1><p id="9015" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">Like any NLP model, the Transformer needs two things about each word — the meaning of the word and its position in the sequence.</p><ul class=""><li id="77ec" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph="">The Embedding layer encodes the meaning of the word.</li><li id="e768" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">The Position Encoding layer represents the position of the word.</li></ul><p id="18be" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Transformer combines these two encodings by adding them.</p><h2 id="bd27" class="pj oh ev be oi pk pl pm ol pn po pp oo nh pq pr ps nl pt pu pv np pw px py fc bj" data-selectable-paragraph="">Embedding</h2><p id="2a4c" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">The Transformer has two Embedding layers. The input sequence is fed to the first Embedding layer, known as the Input Embedding.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*b4MwVq8didNMg9-yD-hXyA.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*b4MwVq8didNMg9-yD-hXyA.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*b4MwVq8didNMg9-yD-hXyA.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*b4MwVq8didNMg9-yD-hXyA.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*b4MwVq8didNMg9-yD-hXyA.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*b4MwVq8didNMg9-yD-hXyA.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*b4MwVq8didNMg9-yD-hXyA.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*b4MwVq8didNMg9-yD-hXyA.png 640w, https://miro.medium.com/v2/resize:fit:720/1*b4MwVq8didNMg9-yD-hXyA.png 720w, https://miro.medium.com/v2/resize:fit:750/1*b4MwVq8didNMg9-yD-hXyA.png 750w, https://miro.medium.com/v2/resize:fit:786/1*b4MwVq8didNMg9-yD-hXyA.png 786w, https://miro.medium.com/v2/resize:fit:828/1*b4MwVq8didNMg9-yD-hXyA.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*b4MwVq8didNMg9-yD-hXyA.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*b4MwVq8didNMg9-yD-hXyA.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="214" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*b4MwVq8didNMg9-yD-hXyA.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="5973" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The target sequence is fed to the second Embedding layer after shifting the targets right by one position and inserting a Start token in the first position. Note that, during Inference, we have no target sequence and we feed the output sequence to this second layer in a loop, as we learned in <a class="af mx" rel="noopener" target="_blank" href="/transformers-explained-visually-part-1-overview-of-functionality-95a6dd460452">Part 1</a>. That is why it is called the Output Embedding.</p><p id="1193" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The text sequence is mapped to numeric word IDs using our vocabulary. The embedding layer then maps each input word into an embedding vector, which is a richer representation of the meaning of that word.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*OEaVrgvL_KN9RyDnWrGLWw.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*OEaVrgvL_KN9RyDnWrGLWw.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*OEaVrgvL_KN9RyDnWrGLWw.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*OEaVrgvL_KN9RyDnWrGLWw.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*OEaVrgvL_KN9RyDnWrGLWw.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*OEaVrgvL_KN9RyDnWrGLWw.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*OEaVrgvL_KN9RyDnWrGLWw.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*OEaVrgvL_KN9RyDnWrGLWw.png 640w, https://miro.medium.com/v2/resize:fit:720/1*OEaVrgvL_KN9RyDnWrGLWw.png 720w, https://miro.medium.com/v2/resize:fit:750/1*OEaVrgvL_KN9RyDnWrGLWw.png 750w, https://miro.medium.com/v2/resize:fit:786/1*OEaVrgvL_KN9RyDnWrGLWw.png 786w, https://miro.medium.com/v2/resize:fit:828/1*OEaVrgvL_KN9RyDnWrGLWw.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*OEaVrgvL_KN9RyDnWrGLWw.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*OEaVrgvL_KN9RyDnWrGLWw.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="409" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*OEaVrgvL_KN9RyDnWrGLWw.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><h2 id="0a83" class="pj oh ev be oi pk pl pm ol pn po pp oo nh pq pr ps nl pt pu pv np pw px py fc bj" data-selectable-paragraph="">Position Encoding</h2><p id="6675" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">Since an RNN implements a loop where each word is input sequentially, it implicitly knows the position of each word.</p><p id="237f" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">However, Transformers don’t use RNNs and all words in a sequence are input in parallel. This is its major advantage over the RNN architecture, but it means that the position information is lost, and has to be added back in separately.</p><p id="edd4" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Just like the two Embedding layers, there are two Position Encoding layers. The Position Encoding is computed independently of the input sequence. These are fixed values that depend only on the max length of the sequence. For instance,</p><ul class=""><li id="f23e" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph="">the first item is a constant code that indicates the first position</li><li id="4e06" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">the second item is a constant code that indicates the second position,</li><li id="5fba" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">and so on.</li></ul><p id="e371" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">These constants are computed using the formula below, where</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div class="mc md pz"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*oIIicuulKrSGVzRM6-tXCA.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*oIIicuulKrSGVzRM6-tXCA.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*oIIicuulKrSGVzRM6-tXCA.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*oIIicuulKrSGVzRM6-tXCA.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*oIIicuulKrSGVzRM6-tXCA.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*oIIicuulKrSGVzRM6-tXCA.png 1100w, https://miro.medium.com/v2/resize:fit:834/format:webp/1*oIIicuulKrSGVzRM6-tXCA.png 834w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 417px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*oIIicuulKrSGVzRM6-tXCA.png 640w, https://miro.medium.com/v2/resize:fit:720/1*oIIicuulKrSGVzRM6-tXCA.png 720w, https://miro.medium.com/v2/resize:fit:750/1*oIIicuulKrSGVzRM6-tXCA.png 750w, https://miro.medium.com/v2/resize:fit:786/1*oIIicuulKrSGVzRM6-tXCA.png 786w, https://miro.medium.com/v2/resize:fit:828/1*oIIicuulKrSGVzRM6-tXCA.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*oIIicuulKrSGVzRM6-tXCA.png 1100w, https://miro.medium.com/v2/resize:fit:834/1*oIIicuulKrSGVzRM6-tXCA.png 834w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 417px"><img alt="" class="bg mq mr c" width="417" height="92" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:626/1*oIIicuulKrSGVzRM6-tXCA.png"></picture></div></figure><ul class=""><li id="335d" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph=""><em class="oa">pos</em> is the position of the word in the sequence</li><li id="a759" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph=""><em class="oa">d_model</em> is the length of the encoding vector (same as the embedding vector) and</li><li id="4d22" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph=""><em class="oa">i</em> is the index value into this vector.</li></ul><figure class="mf mg mh mi mj lt mc md paragraph-image"><div class="mc md qa"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*rBSkDZzEFWFq5u91UPkFHw.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*rBSkDZzEFWFq5u91UPkFHw.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*rBSkDZzEFWFq5u91UPkFHw.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*rBSkDZzEFWFq5u91UPkFHw.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*rBSkDZzEFWFq5u91UPkFHw.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*rBSkDZzEFWFq5u91UPkFHw.png 1100w, https://miro.medium.com/v2/resize:fit:960/format:webp/1*rBSkDZzEFWFq5u91UPkFHw.png 960w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 480px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*rBSkDZzEFWFq5u91UPkFHw.png 640w, https://miro.medium.com/v2/resize:fit:720/1*rBSkDZzEFWFq5u91UPkFHw.png 720w, https://miro.medium.com/v2/resize:fit:750/1*rBSkDZzEFWFq5u91UPkFHw.png 750w, https://miro.medium.com/v2/resize:fit:786/1*rBSkDZzEFWFq5u91UPkFHw.png 786w, https://miro.medium.com/v2/resize:fit:828/1*rBSkDZzEFWFq5u91UPkFHw.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*rBSkDZzEFWFq5u91UPkFHw.png 1100w, https://miro.medium.com/v2/resize:fit:960/1*rBSkDZzEFWFq5u91UPkFHw.png 960w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 480px"><img alt="" class="bg mq mr c" width="480" height="260" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:720/1*rBSkDZzEFWFq5u91UPkFHw.png"></picture></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="5ed2" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">In other words, it interleaves a sine curve and a cos curve, with sine values for all even indexes and cos values for all odd indexes. As an example, if we encode a sequence of 40 words, we can see below the encoding values for a few (word position, encoding_index) combinations.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div class="mc md qb"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*62L4igU75YYId4SuR5wdtA.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*62L4igU75YYId4SuR5wdtA.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*62L4igU75YYId4SuR5wdtA.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*62L4igU75YYId4SuR5wdtA.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*62L4igU75YYId4SuR5wdtA.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*62L4igU75YYId4SuR5wdtA.png 1100w, https://miro.medium.com/v2/resize:fit:860/format:webp/1*62L4igU75YYId4SuR5wdtA.png 860w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 430px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*62L4igU75YYId4SuR5wdtA.png 640w, https://miro.medium.com/v2/resize:fit:720/1*62L4igU75YYId4SuR5wdtA.png 720w, https://miro.medium.com/v2/resize:fit:750/1*62L4igU75YYId4SuR5wdtA.png 750w, https://miro.medium.com/v2/resize:fit:786/1*62L4igU75YYId4SuR5wdtA.png 786w, https://miro.medium.com/v2/resize:fit:828/1*62L4igU75YYId4SuR5wdtA.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*62L4igU75YYId4SuR5wdtA.png 1100w, https://miro.medium.com/v2/resize:fit:860/1*62L4igU75YYId4SuR5wdtA.png 860w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 430px"><img alt="" class="bg mq mr c" width="430" height="273" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:645/1*62L4igU75YYId4SuR5wdtA.png"></picture></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="52c8" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The blue curve shows the encoding of the 0th index for all 40 word-positions and the orange curve shows the encoding of the 1st index for all 40 word-positions. There will be similar curves for the remaining index values.</p><h1 id="719a" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Matrix Dimensions</h1><p id="426f" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">As we know, deep learning models process a batch of training samples at a time. The Embedding and Position Encoding layers operate on matrices representing a batch of sequence samples. The Embedding takes a (samples, sequence length) shaped matrix of word IDs. It encodes each word ID into a word vector whose length is the embedding size, resulting in a (samples, sequence length, embedding size) shaped output matrix. The Position Encoding uses an encoding size that is equal to the embedding size. So it produces a similarly shaped matrix that can be added to the embedding matrix.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*7LPcSJYhSgjPJZWJnp22XA.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*7LPcSJYhSgjPJZWJnp22XA.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*7LPcSJYhSgjPJZWJnp22XA.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*7LPcSJYhSgjPJZWJnp22XA.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*7LPcSJYhSgjPJZWJnp22XA.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*7LPcSJYhSgjPJZWJnp22XA.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*7LPcSJYhSgjPJZWJnp22XA.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*7LPcSJYhSgjPJZWJnp22XA.png 640w, https://miro.medium.com/v2/resize:fit:720/1*7LPcSJYhSgjPJZWJnp22XA.png 720w, https://miro.medium.com/v2/resize:fit:750/1*7LPcSJYhSgjPJZWJnp22XA.png 750w, https://miro.medium.com/v2/resize:fit:786/1*7LPcSJYhSgjPJZWJnp22XA.png 786w, https://miro.medium.com/v2/resize:fit:828/1*7LPcSJYhSgjPJZWJnp22XA.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*7LPcSJYhSgjPJZWJnp22XA.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*7LPcSJYhSgjPJZWJnp22XA.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="506" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*7LPcSJYhSgjPJZWJnp22XA.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="b040" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The (samples, sequence length, embedding size) shape produced by the Embedding and Position Encoding layers is preserved all through the Transformer, as the data flows through the Encoder and Decoder Stacks until it is reshaped by the final Output layers.</p><p id="b098" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">This gives a sense of the 3D matrix dimensions in the Transformer. However, to simplify the visualization, from here on we will drop the first dimension (for the samples) and use the 2D representation for a single sample.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*586QwwjzHY9fP4aVPWrqiQ.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*586QwwjzHY9fP4aVPWrqiQ.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*586QwwjzHY9fP4aVPWrqiQ.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*586QwwjzHY9fP4aVPWrqiQ.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*586QwwjzHY9fP4aVPWrqiQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*586QwwjzHY9fP4aVPWrqiQ.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*586QwwjzHY9fP4aVPWrqiQ.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*586QwwjzHY9fP4aVPWrqiQ.png 640w, https://miro.medium.com/v2/resize:fit:720/1*586QwwjzHY9fP4aVPWrqiQ.png 720w, https://miro.medium.com/v2/resize:fit:750/1*586QwwjzHY9fP4aVPWrqiQ.png 750w, https://miro.medium.com/v2/resize:fit:786/1*586QwwjzHY9fP4aVPWrqiQ.png 786w, https://miro.medium.com/v2/resize:fit:828/1*586QwwjzHY9fP4aVPWrqiQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*586QwwjzHY9fP4aVPWrqiQ.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*586QwwjzHY9fP4aVPWrqiQ.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="355" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*586QwwjzHY9fP4aVPWrqiQ.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="19a2" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Input Embedding sends its outputs into the Encoder. Similarly, the Output Embedding feeds into the Decoder.</p><h1 id="f7dd" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Encoder</h1><p id="1114" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">The Encoder and Decoder Stacks consists of several (usually six) Encoders and Decoders respectively, connected sequentially.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div class="mc md qa"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*Rch3UQW1oHUCrMygAZ_JFQ.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*Rch3UQW1oHUCrMygAZ_JFQ.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*Rch3UQW1oHUCrMygAZ_JFQ.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*Rch3UQW1oHUCrMygAZ_JFQ.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*Rch3UQW1oHUCrMygAZ_JFQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*Rch3UQW1oHUCrMygAZ_JFQ.png 1100w, https://miro.medium.com/v2/resize:fit:960/format:webp/1*Rch3UQW1oHUCrMygAZ_JFQ.png 960w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 480px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*Rch3UQW1oHUCrMygAZ_JFQ.png 640w, https://miro.medium.com/v2/resize:fit:720/1*Rch3UQW1oHUCrMygAZ_JFQ.png 720w, https://miro.medium.com/v2/resize:fit:750/1*Rch3UQW1oHUCrMygAZ_JFQ.png 750w, https://miro.medium.com/v2/resize:fit:786/1*Rch3UQW1oHUCrMygAZ_JFQ.png 786w, https://miro.medium.com/v2/resize:fit:828/1*Rch3UQW1oHUCrMygAZ_JFQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*Rch3UQW1oHUCrMygAZ_JFQ.png 1100w, https://miro.medium.com/v2/resize:fit:960/1*Rch3UQW1oHUCrMygAZ_JFQ.png 960w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 480px"><img alt="" class="bg mq mr c" width="480" height="320" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:720/1*Rch3UQW1oHUCrMygAZ_JFQ.png"></picture></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="dbf7" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The first Encoder in the stack receives its input from the Embedding and Position Encoding. The other Encoders in the stack receive their input from the previous Encoder.</p><p id="f8d9" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Encoder passes its input into a Multi-head Self-attention layer. The Self-attention output is passed into a Feed-forward layer, which then sends its output upwards to the next Encoder.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div class="mc md qc"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*yz-iScQJrwx8c0xB4PWRyQ.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*yz-iScQJrwx8c0xB4PWRyQ.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*yz-iScQJrwx8c0xB4PWRyQ.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*yz-iScQJrwx8c0xB4PWRyQ.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*yz-iScQJrwx8c0xB4PWRyQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*yz-iScQJrwx8c0xB4PWRyQ.png 1100w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*yz-iScQJrwx8c0xB4PWRyQ.png 720w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 360px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*yz-iScQJrwx8c0xB4PWRyQ.png 640w, https://miro.medium.com/v2/resize:fit:720/1*yz-iScQJrwx8c0xB4PWRyQ.png 720w, https://miro.medium.com/v2/resize:fit:750/1*yz-iScQJrwx8c0xB4PWRyQ.png 750w, https://miro.medium.com/v2/resize:fit:786/1*yz-iScQJrwx8c0xB4PWRyQ.png 786w, https://miro.medium.com/v2/resize:fit:828/1*yz-iScQJrwx8c0xB4PWRyQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*yz-iScQJrwx8c0xB4PWRyQ.png 1100w, https://miro.medium.com/v2/resize:fit:720/1*yz-iScQJrwx8c0xB4PWRyQ.png 720w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 360px"><img alt="" class="bg mq mr c" width="360" height="537" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:540/1*yz-iScQJrwx8c0xB4PWRyQ.png"></picture></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="b2b8" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Both the Self-attention and Feed-forward sub-layers, have a residual skip-connection around them, followed by a Layer-Normalization.</p><p id="146d" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The output of the last Encoder is fed into each Decoder in the Decoder Stack as explained below.</p><h1 id="6d28" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Decoder</h1><p id="bfd7" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">The Decoder’s structure is very similar to the Encoder’s but with a couple of differences.</p><p id="0b24" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Like the Encoder, the first Decoder in the stack receives its input from the Output Embedding and Position Encoding. The other Decoders in the stack receive their input from the previous Decoder.</p><p id="ea60" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Decoder passes its input into a Multi-head Self-attention layer. This operates in a slightly different way than the one in the Encoder. It is only allowed to attend to earlier positions in the sequence. This is done by masking future positions, which we’ll talk about shortly.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div class="mc md qc"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*4hOio9qBzUxI3FQsTNUX5w.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*4hOio9qBzUxI3FQsTNUX5w.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*4hOio9qBzUxI3FQsTNUX5w.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*4hOio9qBzUxI3FQsTNUX5w.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*4hOio9qBzUxI3FQsTNUX5w.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*4hOio9qBzUxI3FQsTNUX5w.png 1100w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*4hOio9qBzUxI3FQsTNUX5w.png 720w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 360px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*4hOio9qBzUxI3FQsTNUX5w.png 640w, https://miro.medium.com/v2/resize:fit:720/1*4hOio9qBzUxI3FQsTNUX5w.png 720w, https://miro.medium.com/v2/resize:fit:750/1*4hOio9qBzUxI3FQsTNUX5w.png 750w, https://miro.medium.com/v2/resize:fit:786/1*4hOio9qBzUxI3FQsTNUX5w.png 786w, https://miro.medium.com/v2/resize:fit:828/1*4hOio9qBzUxI3FQsTNUX5w.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*4hOio9qBzUxI3FQsTNUX5w.png 1100w, https://miro.medium.com/v2/resize:fit:720/1*4hOio9qBzUxI3FQsTNUX5w.png 720w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 360px"><img alt="" class="bg mq mr c" width="360" height="500" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:540/1*4hOio9qBzUxI3FQsTNUX5w.png"></picture></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="fd12" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Unlike the Encoder, the Decoder has a second Multi-head attention layer, known as the Encoder-Decoder attention layer. The Encoder-Decoder attention layer works like Self-attention, except that it combines two sources of inputs — the Self-attention layer below it as well as the output of the Encoder stack.</p><p id="7d28" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Self-attention output is passed into a Feed-forward layer, which then sends its output upwards to the next Decoder.</p><p id="401c" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Each of these sub-layers, Self-attention, Encoder-Decoder attention, and Feed-forward, have a residual skip-connection around them, followed by a Layer-Normalization.</p><h1 id="e808" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Attention</h1><p id="1ca1" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">In <a class="af mx" rel="noopener" target="_blank" href="/transformers-explained-visually-part-1-overview-of-functionality-95a6dd460452">Part 1</a>, we talked about why Attention is so important while processing sequences. In the Transformer, Attention is used in three places:</p><ul class=""><li id="bf78" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph="">Self-attention in the Encoder — the input sequence pays attention to itself</li><li id="1a95" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">Self-attention in the Decoder — the target sequence pays attention to itself</li><li id="3aa5" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">Encoder-Decoder-attention in the Decoder — the target sequence pays attention to the input sequence</li></ul><p id="b058" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Attention layer takes its input in the form of three parameters, known as the Query, Key, and Value.</p><ul class=""><li id="b239" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph="">In the Encoder’s Self-attention, the Encoder’s input is passed to all three parameters, Query, Key, and Value.</li></ul><figure class="mf mg mh mi mj lt mc md paragraph-image"><div class="mc md qc"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*4UT-x4vd6FaBCPTy_Y74yA.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*4UT-x4vd6FaBCPTy_Y74yA.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*4UT-x4vd6FaBCPTy_Y74yA.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*4UT-x4vd6FaBCPTy_Y74yA.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*4UT-x4vd6FaBCPTy_Y74yA.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*4UT-x4vd6FaBCPTy_Y74yA.png 1100w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*4UT-x4vd6FaBCPTy_Y74yA.png 720w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 360px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*4UT-x4vd6FaBCPTy_Y74yA.png 640w, https://miro.medium.com/v2/resize:fit:720/1*4UT-x4vd6FaBCPTy_Y74yA.png 720w, https://miro.medium.com/v2/resize:fit:750/1*4UT-x4vd6FaBCPTy_Y74yA.png 750w, https://miro.medium.com/v2/resize:fit:786/1*4UT-x4vd6FaBCPTy_Y74yA.png 786w, https://miro.medium.com/v2/resize:fit:828/1*4UT-x4vd6FaBCPTy_Y74yA.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*4UT-x4vd6FaBCPTy_Y74yA.png 1100w, https://miro.medium.com/v2/resize:fit:720/1*4UT-x4vd6FaBCPTy_Y74yA.png 720w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 360px"><img alt="" class="bg mq mr c" width="360" height="560" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:540/1*4UT-x4vd6FaBCPTy_Y74yA.png"></picture></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><ul class=""><li id="ed1d" class="my mz ev na b ga nb nc nd gd ne nf ng nu ni nj nk nv nm nn no nw nq nr ns nt pi ny nz bj" data-selectable-paragraph="">In the Decoder’s Self-attention, the Decoder’s input is passed to all three parameters, Query, Key, and Value.</li><li id="8f24" class="my mz ev na b ga ob nc nd gd oc nf ng nu od nj nk nv oe nn no nw of nr ns nt pi ny nz bj" data-selectable-paragraph="">In the Decoder’s Encoder-Decoder attention, the output of the final Encoder in the stack is passed to the Value and Key parameters. The output of the Self-attention (and Layer Norm) module below it is passed to the Query parameter.</li></ul><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*nEHe_LpPado5b8FRIcnTHQ.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*nEHe_LpPado5b8FRIcnTHQ.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*nEHe_LpPado5b8FRIcnTHQ.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*nEHe_LpPado5b8FRIcnTHQ.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*nEHe_LpPado5b8FRIcnTHQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*nEHe_LpPado5b8FRIcnTHQ.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*nEHe_LpPado5b8FRIcnTHQ.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*nEHe_LpPado5b8FRIcnTHQ.png 640w, https://miro.medium.com/v2/resize:fit:720/1*nEHe_LpPado5b8FRIcnTHQ.png 720w, https://miro.medium.com/v2/resize:fit:750/1*nEHe_LpPado5b8FRIcnTHQ.png 750w, https://miro.medium.com/v2/resize:fit:786/1*nEHe_LpPado5b8FRIcnTHQ.png 786w, https://miro.medium.com/v2/resize:fit:828/1*nEHe_LpPado5b8FRIcnTHQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*nEHe_LpPado5b8FRIcnTHQ.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*nEHe_LpPado5b8FRIcnTHQ.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="628" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*nEHe_LpPado5b8FRIcnTHQ.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><h1 id="9c6a" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Multi-head Attention</h1><p id="f186" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">The Transformer calls each Attention processor an Attention Head and repeats it several times in parallel. This is known as Multi-head attention. It gives its Attention greater power of discrimination, by combining several similar Attention calculations.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*DKNIOlVfbh9K1EqU5iDJKA.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*DKNIOlVfbh9K1EqU5iDJKA.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*DKNIOlVfbh9K1EqU5iDJKA.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*DKNIOlVfbh9K1EqU5iDJKA.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*DKNIOlVfbh9K1EqU5iDJKA.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*DKNIOlVfbh9K1EqU5iDJKA.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*DKNIOlVfbh9K1EqU5iDJKA.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*DKNIOlVfbh9K1EqU5iDJKA.png 640w, https://miro.medium.com/v2/resize:fit:720/1*DKNIOlVfbh9K1EqU5iDJKA.png 720w, https://miro.medium.com/v2/resize:fit:750/1*DKNIOlVfbh9K1EqU5iDJKA.png 750w, https://miro.medium.com/v2/resize:fit:786/1*DKNIOlVfbh9K1EqU5iDJKA.png 786w, https://miro.medium.com/v2/resize:fit:828/1*DKNIOlVfbh9K1EqU5iDJKA.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*DKNIOlVfbh9K1EqU5iDJKA.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*DKNIOlVfbh9K1EqU5iDJKA.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="564" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*DKNIOlVfbh9K1EqU5iDJKA.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="3bdd" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Query, Key, and Value are each passed through separate Linear layers, each with their own weights, producing three results called Q, K, and V respectively. These are then combined together using the Attention formula as shown below, to produce the Attention Score.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div class="mc md qd"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*z01aP_2l3OowwRVAeQH8Xw.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*z01aP_2l3OowwRVAeQH8Xw.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*z01aP_2l3OowwRVAeQH8Xw.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*z01aP_2l3OowwRVAeQH8Xw.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*z01aP_2l3OowwRVAeQH8Xw.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*z01aP_2l3OowwRVAeQH8Xw.png 1100w, https://miro.medium.com/v2/resize:fit:1284/format:webp/1*z01aP_2l3OowwRVAeQH8Xw.png 1284w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 642px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*z01aP_2l3OowwRVAeQH8Xw.png 640w, https://miro.medium.com/v2/resize:fit:720/1*z01aP_2l3OowwRVAeQH8Xw.png 720w, https://miro.medium.com/v2/resize:fit:750/1*z01aP_2l3OowwRVAeQH8Xw.png 750w, https://miro.medium.com/v2/resize:fit:786/1*z01aP_2l3OowwRVAeQH8Xw.png 786w, https://miro.medium.com/v2/resize:fit:828/1*z01aP_2l3OowwRVAeQH8Xw.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*z01aP_2l3OowwRVAeQH8Xw.png 1100w, https://miro.medium.com/v2/resize:fit:1284/1*z01aP_2l3OowwRVAeQH8Xw.png 1284w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 642px"><img alt="" class="bg mq mr c" width="642" height="685" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:963/1*z01aP_2l3OowwRVAeQH8Xw.png"></picture></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="ecf5" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The important thing to realize here is that the Q, K, and V values carry an encoded representation of each word in the sequence. The Attention calculations then combine each word with every other word in the sequence, so that the Attention Score encodes a score for each word in the sequence.</p><p id="6eb8" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">When discussing the Decoder a little while back, we briefly mentioned masking. The Mask is also shown in the Attention diagrams above. Let’s see how it works.</p><h1 id="e7eb" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Attention Masks</h1><p id="5aeb" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">While computing the Attention Score, the Attention module implements a masking step. Masking serves two purposes:</p><p id="2a88" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph=""><strong class="na fh">In the Encoder Self-attention and in the Encoder-Decoder-attention:</strong> masking serves to zero attention outputs where there is padding in the input sentences, to ensure that padding doesn’t contribute to the self-attention. (Note: since input sequences could be of different lengths they are extended with padding tokens like in most NLP applications so that fixed-length vectors can be input to the Transformer.)</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*026nzf4bw_DdSZ7kLJWcog.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*026nzf4bw_DdSZ7kLJWcog.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*026nzf4bw_DdSZ7kLJWcog.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*026nzf4bw_DdSZ7kLJWcog.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*026nzf4bw_DdSZ7kLJWcog.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*026nzf4bw_DdSZ7kLJWcog.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*026nzf4bw_DdSZ7kLJWcog.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*026nzf4bw_DdSZ7kLJWcog.png 640w, https://miro.medium.com/v2/resize:fit:720/1*026nzf4bw_DdSZ7kLJWcog.png 720w, https://miro.medium.com/v2/resize:fit:750/1*026nzf4bw_DdSZ7kLJWcog.png 750w, https://miro.medium.com/v2/resize:fit:786/1*026nzf4bw_DdSZ7kLJWcog.png 786w, https://miro.medium.com/v2/resize:fit:828/1*026nzf4bw_DdSZ7kLJWcog.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*026nzf4bw_DdSZ7kLJWcog.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*026nzf4bw_DdSZ7kLJWcog.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="302" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*026nzf4bw_DdSZ7kLJWcog.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="53fd" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Similarly for the Encoder-Decoder attention.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*QxJ5ySWdZgDOUezj5iOhdg.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*QxJ5ySWdZgDOUezj5iOhdg.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*QxJ5ySWdZgDOUezj5iOhdg.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*QxJ5ySWdZgDOUezj5iOhdg.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*QxJ5ySWdZgDOUezj5iOhdg.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*QxJ5ySWdZgDOUezj5iOhdg.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*QxJ5ySWdZgDOUezj5iOhdg.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*QxJ5ySWdZgDOUezj5iOhdg.png 640w, https://miro.medium.com/v2/resize:fit:720/1*QxJ5ySWdZgDOUezj5iOhdg.png 720w, https://miro.medium.com/v2/resize:fit:750/1*QxJ5ySWdZgDOUezj5iOhdg.png 750w, https://miro.medium.com/v2/resize:fit:786/1*QxJ5ySWdZgDOUezj5iOhdg.png 786w, https://miro.medium.com/v2/resize:fit:828/1*QxJ5ySWdZgDOUezj5iOhdg.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*QxJ5ySWdZgDOUezj5iOhdg.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*QxJ5ySWdZgDOUezj5iOhdg.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="282" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*QxJ5ySWdZgDOUezj5iOhdg.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="d245" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph=""><mark class="ady adz ao"><strong class="na fh">In the Decoder Self-attention: </strong></mark><mark class="ady adz ao">masking serves to prevent the decoder from ‘peeking’ ahead at the rest of the target sentence when predicting the next word.</mark></p><p id="e06a" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Decoder processes words in the source sequence and uses them to predict the words in the destination sequence. During training, this is done via Teacher Forcing, where the complete target sequence is fed as Decoder inputs. Therefore, while predicting a word at a certain position, the Decoder has available to it the target words preceding that word as well as the target words following that word. This allows the Decoder to ‘cheat’ by using target words from future ‘time steps’.</p><p id="905d" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">For instance, when predicting ‘<em class="oa">Word 3’</em>, the Decoder should refer only to the first 3 input words from the target but not the fourth word ‘<em class="oa">Ketan’</em>.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*H0r38u_sLfDUBKgeVg4ssA.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*H0r38u_sLfDUBKgeVg4ssA.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*H0r38u_sLfDUBKgeVg4ssA.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*H0r38u_sLfDUBKgeVg4ssA.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*H0r38u_sLfDUBKgeVg4ssA.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*H0r38u_sLfDUBKgeVg4ssA.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*H0r38u_sLfDUBKgeVg4ssA.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*H0r38u_sLfDUBKgeVg4ssA.png 640w, https://miro.medium.com/v2/resize:fit:720/1*H0r38u_sLfDUBKgeVg4ssA.png 720w, https://miro.medium.com/v2/resize:fit:750/1*H0r38u_sLfDUBKgeVg4ssA.png 750w, https://miro.medium.com/v2/resize:fit:786/1*H0r38u_sLfDUBKgeVg4ssA.png 786w, https://miro.medium.com/v2/resize:fit:828/1*H0r38u_sLfDUBKgeVg4ssA.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*H0r38u_sLfDUBKgeVg4ssA.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*H0r38u_sLfDUBKgeVg4ssA.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="195" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*H0r38u_sLfDUBKgeVg4ssA.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="c765" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Therefore, the Decoder masks out input words that appear later in the sequence.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*cawtdZLjT9hp7ByG2vcKOQ.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*cawtdZLjT9hp7ByG2vcKOQ.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*cawtdZLjT9hp7ByG2vcKOQ.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*cawtdZLjT9hp7ByG2vcKOQ.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*cawtdZLjT9hp7ByG2vcKOQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*cawtdZLjT9hp7ByG2vcKOQ.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*cawtdZLjT9hp7ByG2vcKOQ.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*cawtdZLjT9hp7ByG2vcKOQ.png 640w, https://miro.medium.com/v2/resize:fit:720/1*cawtdZLjT9hp7ByG2vcKOQ.png 720w, https://miro.medium.com/v2/resize:fit:750/1*cawtdZLjT9hp7ByG2vcKOQ.png 750w, https://miro.medium.com/v2/resize:fit:786/1*cawtdZLjT9hp7ByG2vcKOQ.png 786w, https://miro.medium.com/v2/resize:fit:828/1*cawtdZLjT9hp7ByG2vcKOQ.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*cawtdZLjT9hp7ByG2vcKOQ.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*cawtdZLjT9hp7ByG2vcKOQ.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="282" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*cawtdZLjT9hp7ByG2vcKOQ.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="553a" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">When calculating the Attention Score (refer to the picture earlier showing the calculations) masking is applied to the numerator just before the Softmax. The masked out elements (white squares) are set to negative infinity, so that Softmax turns those values to zero.</p><h1 id="ac13" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Generate Output</h1><p id="3f98" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">The last Decoder in the stack passes its output to the Output component which converts it into the final output sentence.</p><p id="3c18" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Linear layer projects the Decoder vector into Word Scores, with a score value for each unique word in the target vocabulary, at each position in the sentence. For instance, if our final output sentence has 7 words and the target Spanish vocabulary has 10000 unique words, we generate 10000 score values for each of those 7 words. The score values indicate the likelihood of occurrence for each word in the vocabulary in that position of the sentence.</p><p id="2643" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Softmax layer then turns those scores into probabilities (which add up to 1.0). In each position, we find the index for the word with the highest probability, and then map that index to the corresponding word in the vocabulary. Those words then form the output sequence of the Transformer.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*dQTK3oeYqOBUDVgNktSpCw.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*dQTK3oeYqOBUDVgNktSpCw.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*dQTK3oeYqOBUDVgNktSpCw.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*dQTK3oeYqOBUDVgNktSpCw.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*dQTK3oeYqOBUDVgNktSpCw.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*dQTK3oeYqOBUDVgNktSpCw.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*dQTK3oeYqOBUDVgNktSpCw.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*dQTK3oeYqOBUDVgNktSpCw.png 640w, https://miro.medium.com/v2/resize:fit:720/1*dQTK3oeYqOBUDVgNktSpCw.png 720w, https://miro.medium.com/v2/resize:fit:750/1*dQTK3oeYqOBUDVgNktSpCw.png 750w, https://miro.medium.com/v2/resize:fit:786/1*dQTK3oeYqOBUDVgNktSpCw.png 786w, https://miro.medium.com/v2/resize:fit:828/1*dQTK3oeYqOBUDVgNktSpCw.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*dQTK3oeYqOBUDVgNktSpCw.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*dQTK3oeYqOBUDVgNktSpCw.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="642" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*dQTK3oeYqOBUDVgNktSpCw.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><h1 id="a58f" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Training and Loss Function</h1><p id="e72b" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">During training, we use a loss function such as cross-entropy loss to compare the generated output probability distribution to the target sequence. The probability distribution gives the probability of each word occurring in that position.</p><figure class="mf mg mh mi mj lt mc md paragraph-image"><div role="button" tabindex="0" class="mm mn hh mo bg mp"><div class="mc md ph"><picture><source srcset="https://miro.medium.com/v2/resize:fit:640/format:webp/1*50j_urNlRnctglsXCRoeiw.png 640w, https://miro.medium.com/v2/resize:fit:720/format:webp/1*50j_urNlRnctglsXCRoeiw.png 720w, https://miro.medium.com/v2/resize:fit:750/format:webp/1*50j_urNlRnctglsXCRoeiw.png 750w, https://miro.medium.com/v2/resize:fit:786/format:webp/1*50j_urNlRnctglsXCRoeiw.png 786w, https://miro.medium.com/v2/resize:fit:828/format:webp/1*50j_urNlRnctglsXCRoeiw.png 828w, https://miro.medium.com/v2/resize:fit:1100/format:webp/1*50j_urNlRnctglsXCRoeiw.png 1100w, https://miro.medium.com/v2/resize:fit:1400/format:webp/1*50j_urNlRnctglsXCRoeiw.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px" type="image/webp"><source data-testid="og" srcset="https://miro.medium.com/v2/resize:fit:640/1*50j_urNlRnctglsXCRoeiw.png 640w, https://miro.medium.com/v2/resize:fit:720/1*50j_urNlRnctglsXCRoeiw.png 720w, https://miro.medium.com/v2/resize:fit:750/1*50j_urNlRnctglsXCRoeiw.png 750w, https://miro.medium.com/v2/resize:fit:786/1*50j_urNlRnctglsXCRoeiw.png 786w, https://miro.medium.com/v2/resize:fit:828/1*50j_urNlRnctglsXCRoeiw.png 828w, https://miro.medium.com/v2/resize:fit:1100/1*50j_urNlRnctglsXCRoeiw.png 1100w, https://miro.medium.com/v2/resize:fit:1400/1*50j_urNlRnctglsXCRoeiw.png 1400w" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw, 700px"><img alt="" class="bg mq mr c" width="700" height="321" loading="lazy" role="presentation" src="https://miro.medium.com/v2/resize:fit:1050/1*50j_urNlRnctglsXCRoeiw.png"></picture></div></div><figcaption class="ms mt mu mc md mv mw be b bf z fd" data-selectable-paragraph="">(Image by Author)</figcaption></figure><p id="04f2" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">Let’s assume our target vocabulary contains just four words. Our goal is to produce a probability distribution that matches our expected target sequence “De nada END”.</p><p id="ed0a" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">This means that the probability distribution for the first word-position should have a probability of 1 for “De” with probabilities for all other words in the vocabulary being 0. Similarly, “nada” and “END” should have a probability of 1 for the second and third word-positions respectively.</p><p id="1b65" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">As usual, the loss is used to compute gradients to train the Transformer via backpropagation.</p><h1 id="ce27" class="og oh ev be oi oj ok gc ol om on gf oo op oq or os ot ou ov ow ox oy oz pa pb bj" data-selectable-paragraph="">Conclusion</h1><p id="3c2e" class="pw-post-body-paragraph my mz ev na b ga pc nc nd gd pd nf ng nh pe nj nk nl pf nn no np pg nr ns nt eo bj" data-selectable-paragraph="">Hopefully, this gives you a feel for what goes on inside the Transformer during Training. As we discussed in the previous article, it runs in a loop during Inference but most of the processing remains the same.</p><p id="dc61" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">The Multi-head Attention module is what gives the Transformer its power. In the next article, we will continue our journey and go one step deeper to really understand the details of how Attention is computed.</p><p id="4f2f" class="pw-post-body-paragraph my mz ev na b ga nb nc nd gd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt eo bj" data-selectable-paragraph="">And finally, if you liked this article, you might also enjoy my other series on Audio Deep Learning, Geolocation Machine Learning, and Image Caption architectures.</p></div></div>